# TASK 1: Getting Data From AlphaVantage API and Cleaning

## Getting Data From AlphaVantage

The below code is used to get the financial data from the AlphaVantage API. You can run this code to get the data for any stock. However, you need to have an API key from AlphaVantage. You can get a free API key from https://www.alphavantage.co/support/#api-key. However, the data has already been extracted and stored in the financial_statements folder, thus you can skip the below code cell and run the next one directly as we have already uploaded the data on github as well.

In [ ]:
import pandas as pd
import requests
import os

# Base URL for Alpha Vantage API
base_url = "https://www.alphavantage.co/query"
# api_key = ""  # Replace with your actual API key


stock_symbols = ["MSFT", "NVDA", "GS", "GME", "GOOGL"]


output_folder = "financial_statements"
os.makedirs(output_folder, exist_ok=True)

# Function to fetch quarterly financial data
def fetch_financial_data(symbol, function):
    params = {
        "function": function,
        "symbol": symbol,
        "apikey": api_key
    }
    response = requests.get(base_url, params=params)
    data = response.json()

    # Extract "quarterlyReports" data if available
    if "quarterlyReports" in data:
        df = pd.DataFrame(data["quarterlyReports"])
    else:
        df = pd.DataFrame()  

    return df

# Loop through each stock symbol and fetch financial statements
for symbol in stock_symbols:
    print(f"Fetching financial data for {symbol}...")

    # Fetch financial data for each report type
    income_df = fetch_financial_data(symbol, "INCOME_STATEMENT")
    balance_df = fetch_financial_data(symbol, "BALANCE_SHEET")
    cashflow_df = fetch_financial_data(symbol, "CASH_FLOW")
    

    
    if not income_df.empty:
        income_df.to_csv(f"{output_folder}/{symbol}_income_statement.csv", index=False)

    if not balance_df.empty:
        balance_df.to_csv(f"{output_folder}/{symbol}_balance_sheet.csv", index=False)

    if not cashflow_df.empty:
        cashflow_df.to_csv(f"{output_folder}/{symbol}_cash_flow.csv", index=False)


    print(f"Financial data for {symbol} saved successfully!")

print("All financial statements have been saved.")

## Two functions to clean data

In [31]:
import pandas as pd
import os

# List of stock symbols to process
stock_symbols = ["GME", "GS", "MSFT", "NVDA", "GOOGL"]

# Directories for input CSVs and output merged CSVs
financial_dir = "financial_statements"       # where individual CSVs are stored
merged_dir = "merged_financials"             # folder for intermediate merged outputs
cleaned_dir = "merged_financials_cleaned"      # folder for final cleaned outputs


os.makedirs(merged_dir, exist_ok=True)
os.makedirs(cleaned_dir, exist_ok=True)


cutoff = 0.10

def clean_individual_df(df):
    """
    Cleans a DataFrame by:
      - Dropping columns with > cutoff missing values (except the date column).
      - Converting the fiscal date column (if present) to datetime and renaming it to "date".
      - Forward-filling missing values.
      - Converting numeric columns (excluding "date") to int64 if possible.
    """
    
    missing_prcnt = df.isnull().mean().sort_values(ascending=False)
    cols_to_drop = list(missing_prcnt[missing_prcnt > cutoff].index)
    if 'fiscalDateEnding' in cols_to_drop:
        cols_to_drop.remove('fiscalDateEnding')
    df.drop(columns=cols_to_drop, inplace=True)
    
    
    if 'fiscalDateEnding' in df.columns:
        df['date'] = pd.to_datetime(df['fiscalDateEnding'])
        df.drop(columns=['fiscalDateEnding'], inplace=True)
    
    elif 'date' in df.columns:
        df['date'] = pd.to_datetime(df['date'])
    
    
    df = df.ffill()
    
    
    for col in df.columns:
        if col != 'date':
            try:
                # We use pd.to_numeric to ensure numeric conversion; if it fails, the column remains unchanged.
                df[col] = pd.to_numeric(df[col], errors='raise').astype('int64')
            except Exception:
                pass
    return df

def final_cleaning(df):
    """
    Final cleaning: Convert all columns (except "date") to numeric and fill any remaining NaNs.
    """
    for col in df.columns:
        if col != 'date':
            df[col] = pd.to_numeric(df[col], errors='coerce')
    df = df.fillna(method='ffill').fillna(method='bfill')
    if df.isnull().sum().sum() > 0:
        print("Warning: There are still missing values after final cleaning!")
    return df


for symbol in stock_symbols:
    print(f"Processing TASK 1 for {symbol}...")

    # Build file paths for the three financial statement CSVs
    income_file = os.path.join(financial_dir, f"{symbol}_income_statement.csv")
    balance_file = os.path.join(financial_dir, f"{symbol}_balance_sheet.csv")
    cashflow_file = os.path.join(financial_dir, f"{symbol}_cash_flow.csv")
    
    
    income_df = pd.read_csv(income_file)
    balance_df = pd.read_csv(balance_file)
    cashflow_df = pd.read_csv(cashflow_file)
    
    
    drop_cols = ['Unnamed: 0', 'reportedCurrency']
    income_df = income_df.drop(columns=drop_cols, errors='ignore')
    balance_df = balance_df.drop(columns=drop_cols, errors='ignore')
    cashflow_df = cashflow_df.drop(columns=drop_cols, errors='ignore')
    
    # Clean each individual DataFrame
    income_df = clean_individual_df(income_df)
    balance_df = clean_individual_df(balance_df)
    cashflow_df = clean_individual_df(cashflow_df)
    
    
    merged_df = pd.merge(pd.merge(balance_df, income_df, on="date", how="inner"),
                         cashflow_df, on="date", how="inner")
    
    # Remove duplicate columns that might have been created during the merge (e.g., with _x and _y suffixes)
    for col in merged_df.columns:
        if '_x' in col:
            base_col = col.replace('_x', '')
            if base_col + '_y' in merged_df.columns:
                merged_df.drop(columns=[base_col + '_y'], inplace=True)
            merged_df.rename(columns={col: base_col}, inplace=True)
    
    # Save the intermediate merged DataFrame
    merged_output_file = os.path.join(merged_dir, f"{symbol}_merged_financials.csv")
    merged_df.to_csv(merged_output_file, index=False)
    
    # Final cleaning to ensure no missing or weird values remain
    merged_df_cleaned = final_cleaning(merged_df)
    
    
    cleaned_output_file = os.path.join(cleaned_dir, f"{symbol}_merged_financials_cleaned.csv")
    merged_df_cleaned.to_csv(cleaned_output_file, index=False)
    print(f"{symbol}: Merged file saved to {merged_output_file}")
    print(f"{symbol}: Final cleaned file saved to {cleaned_output_file}\n")

print("TASK 1 completed for all stock symbols using date as the merge key.")

Processing TASK 1 for GME...
GME: Merged file saved to merged_financials/GME_merged_financials.csv
GME: Final cleaned file saved to merged_financials_cleaned/GME_merged_financials_cleaned.csv

Processing TASK 1 for GS...
GS: Merged file saved to merged_financials/GS_merged_financials.csv
GS: Final cleaned file saved to merged_financials_cleaned/GS_merged_financials_cleaned.csv

Processing TASK 1 for MSFT...
MSFT: Merged file saved to merged_financials/MSFT_merged_financials.csv
MSFT: Final cleaned file saved to merged_financials_cleaned/MSFT_merged_financials_cleaned.csv

Processing TASK 1 for NVDA...
NVDA: Merged file saved to merged_financials/NVDA_merged_financials.csv
NVDA: Final cleaned file saved to merged_financials_cleaned/NVDA_merged_financials_cleaned.csv

Processing TASK 1 for GOOGL...
GOOGL: Merged file saved to merged_financials/GOOGL_merged_financials.csv
GOOGL: Final cleaned file saved to merged_financials_cleaned/GOOGL_merged_financials_cleaned.csv

TASK 1 completed for

/var/folders/cg/lqjpnn456tdcqmv7z04c7lch0000gn/T/ipykernel_79329/3735780683.py:62: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='ffill').fillna(method='bfill')
/var/folders/cg/lqjpnn456tdcqmv7z04c7lch0000gn/T/ipykernel_79329/3735780683.py:62: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='ffill').fillna(method='bfill')
/var/folders/cg/lqjpnn456tdcqmv7z04c7lch0000gn/T/ipykernel_79329/3735780683.py:62: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='ffill').fillna(method='bfill')
/var/folders/cg/lqjpnn456tdcqmv7z04c7lch0000gn/T/ipykernel_79329/3735780683.py:62: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future versi

## Checking to See if No Missing Values

In [32]:
import pandas as pd
import os


stock_symbols = ["GME", "GOOGL", "GS", "MSFT", "NVDA"]


cleaned_dir = "merged_financials_cleaned"

for symbol in stock_symbols:
    file_path = os.path.join(cleaned_dir, f"{symbol}_merged_financials_cleaned.csv")
    try:
        df = pd.read_csv(file_path)
        missing_count = df.isnull().sum().sum()
        if missing_count == 0:
            print(f"{symbol}: No missing values found!")
        else:
            print(f"{symbol}: Warning - {missing_count} missing values detected!")
    except Exception as e:
        print(f"Error processing {symbol} at {file_path}: {e}")

GME: No missing values found!
GOOGL: No missing values found!
GS: No missing values found!
MSFT: No missing values found!
NVDA: No missing values found!


## Checking if the date column is present in the merged financials

In [33]:
import pandas as pd
import os


stock_symbols = ["GME", "GS", "MSFT", "NVDA", "GOOGL"]

merged_df = pd.read_csv("merged_financials_cleaned/NVDA_merged_financials_cleaned.csv")

merged_df['date']

0     2024-10-27
1     2024-07-28
2     2024-04-28
3     2024-01-28
4     2023-10-29
         ...    
56    2010-10-31
57    2010-08-01
58    2010-05-02
59    2010-01-31
60    2009-10-25
Name: date, Length: 61, dtype: object

# TASK 2: Getting Earnings Dates, EPS Estimates, Reported EPS, Suprise % and Merging 

In [34]:
import os
import pandas as pd
import yfinance as yf


stock_symbols = ["GME", "GS", "MSFT", "NVDA", "GOOGL"]

# Directories for cleaned merged CSVs and for saving final merged outputs for Task 2
cleaned_dir = "merged_financials_cleaned"
final_dir = "final_finance_data"
os.makedirs(final_dir, exist_ok=True)

# Number of earnings dates to fetch 
n_earnings = 64

for symbol in stock_symbols:
    print(f"Processing TASK 2 for {symbol}...")
    
    # Create a yfinance Ticker object and fetch earnings dates
    ticker = yf.Ticker(symbol)
    earnings_df = ticker.get_earnings_dates(n_earnings)
    
    # Filter out any future dates
    today = pd.Timestamp.now(tz='America/New_York').normalize()
    earnings_df = earnings_df[earnings_df.index <= today]
    
    # Reset index and rename the date column to "date"
    earnings_df = earnings_df.reset_index()
    earnings_df.rename(columns={earnings_df.columns[0]: 'date'}, inplace=True)
    earnings_df['date'] = pd.to_datetime(earnings_df['date'])
    # Remove timezone information to make the date column timezone-naive
    earnings_df['date'] = earnings_df['date'].dt.tz_localize(None)
    
    
    cleaned_file = os.path.join(cleaned_dir, f"{symbol}_merged_financials_cleaned.csv")
    try:
        finance_df = pd.read_csv(cleaned_file)
    except Exception as e:
        print(f"Error reading {cleaned_file}: {e}")
        continue
    
    
    finance_df['date'] = pd.to_datetime(finance_df['date']).dt.tz_localize(None)
    
    # Sort both DataFrames by "date" (required for merge_asof)
    finance_df.sort_values(by='date', inplace=True)
    earnings_df.sort_values(by='date', inplace=True)
    
    # Merge the financial data with the earnings data using merge_asof
    # Here, we assume that the earnings announcement happens after the quarter end.
    final_df = pd.merge_asof(finance_df, earnings_df, on='date', direction='forward')
    
    
    final_file = os.path.join(final_dir, f"{symbol}_final_finance_data.csv")
    final_df.to_csv(final_file, index=False)
    print(f"{symbol}: Final merged file saved to {final_file}\n")

print("TASK 2 completed for all stock symbols using date as the merge key.")

Processing TASK 2 for GME...
GME: Final merged file saved to final_finance_data/GME_final_finance_data.csv

Processing TASK 2 for GS...
GS: Final merged file saved to final_finance_data/GS_final_finance_data.csv

Processing TASK 2 for MSFT...
MSFT: Final merged file saved to final_finance_data/MSFT_final_finance_data.csv

Processing TASK 2 for NVDA...
NVDA: Final merged file saved to final_finance_data/NVDA_final_finance_data.csv

Processing TASK 2 for GOOGL...
GOOGL: Final merged file saved to final_finance_data/GOOGL_final_finance_data.csv

TASK 2 completed for all stock symbols using date as the merge key.


## TASK 3: Checking for Multi-Collinearity and Shifting EPS by a Quarter to Make Accurate Predictions

In [35]:
import pandas as pd
import os
import numpy as np
from statsmodels.stats.outliers_influence import variance_inflation_factor


stock_symbols = ["GME", "GS", "MSFT", "NVDA", "GOOGL"]


final_dir = "final_finance_data"       # From TASK 2
regression_dir = "regression_data"       # Where regression-ready data will be saved
os.makedirs(regression_dir, exist_ok=True)


threshold_corr = 0.10   
vif_threshold = 10.0    
target_column = 'Surprise(%)'

def calculate_vif(df, features):
    """
    Calculate VIF for a set of features in df.
    Force numeric conversion, replace inf and NaNs, then compute VIF.
    """
    X = df[features].copy().apply(pd.to_numeric, errors='coerce')
    X.replace([np.inf, -np.inf], np.nan, inplace=True)
    X = X.fillna(X.mean())
    X['constant'] = 1
    vif = pd.Series(
        [variance_inflation_factor(X.values, i) for i in range(X.shape[1]-1)],
        index=features
    )
    return vif

def reduce_multicollinearity(df, features, vif_threshold):
    """
    Iteratively remove the feature with the highest VIF until all features are below vif_threshold.
    """
    features = features.copy()
    while True:
        vif = calculate_vif(df, features)
        max_vif = vif.max()
        if max_vif > vif_threshold:
            feature_to_drop = vif.idxmax()
            print(f"   Dropping '{feature_to_drop}' with VIF = {max_vif:.2f}")
            features.remove(feature_to_drop)
        else:
            break
    return features, calculate_vif(df, features)

for symbol in stock_symbols:
    file_path = os.path.join(final_dir, f"{symbol}_final_finance_data.csv")
    try:
        finance_df = pd.read_csv(file_path)
    except Exception as e:
        print(f"Error reading file for {symbol}: {e}")
        continue

    # Convert "date" to datetime, sort descending, then drop "date" for regression purposes
    if 'date' in finance_df.columns:
        finance_df['date'] = pd.to_datetime(finance_df['date'])
        finance_df.sort_values(by='date', ascending=False, inplace=True)
        # Keep a copy of date if needed, but drop from the regression dataset
        finance_df = finance_df.drop(columns=['date'])
    else:
        print(f"Warning: No date column found for {symbol}")

    # Compute correlations (only numeric columns)
    try:
        correlations = finance_df.corr()[target_column].drop(target_column)
    except Exception as e:
        print(f"Error computing correlations for {symbol}: {e}")
        continue

    
    sorted_corr = correlations.abs().sort_values(ascending=False)
    top_labels = sorted_corr[sorted_corr > threshold_corr].index.tolist()
    
    # Fallback: if fewer than 3 features remain, use all available numeric columns (exclude target, Reported EPS)
    numeric_cols = finance_df.select_dtypes(include=[np.number]).columns.tolist()
    fallback_features = [col for col in numeric_cols if col not in [target_column, 'Reported EPS']]
    if len(top_labels) < 3:
        print(f"{symbol}: Only {len(top_labels)} features passed correlation filter. Using fallback features.")
        top_labels = fallback_features
    
    print(f"\n{symbol}:")
    print("   Features correlated with target (>|{0}|):".format(threshold_corr), top_labels)
    
   
    selected_columns = [target_column] + top_labels + ['Reported EPS']
    selected_columns = list(dict.fromkeys(selected_columns))
    
    if 'Reported EPS' not in finance_df.columns:
        print(f"   'Reported EPS' column not found for {symbol}. Skipping.")
        continue
    reduced_data = finance_df[selected_columns].copy()
    
    
    candidate_features = [f for f in top_labels if f != 'Reported EPS']
    
    
    final_features, vif_series = reduce_multicollinearity(reduced_data, candidate_features, vif_threshold)
    print("   Final independent features after VIF filtering:", final_features)
    print("   VIF values:\n", vif_series, "\n")
    
    
    regression_columns = [target_column] + final_features + ['Reported EPS']
    reduced_data = reduced_data[regression_columns].copy()
    
    
    reduced_data['Reported EPS'] = reduced_data['Reported EPS'].fillna(method='ffill').fillna(method='bfill')
    
    
    reduced_data['Shift_Reported_EPS'] = reduced_data['Reported EPS'].shift(1)
    
    # Drop the first row (which will have NaN in Shift_Reported_EPS due to the shift)
    reduced_data = reduced_data.drop(index=reduced_data.index[0])
    
    
    if reduced_data['Shift_Reported_EPS'].isnull().any():
        print(f"Warning: {symbol} still has NaN in Shift_Reported_EPS after shifting.")
    
    
    reduced_data.drop(columns=['Reported EPS'], inplace=True)
    
    
    output_file = os.path.join(regression_dir, f"{symbol}_regression_data.csv")
    reduced_data.to_csv(output_file, index=False)
    print(f"   Regression data for {symbol} saved to {output_file}")

print("\nTASK 3 completed for all stock symbols with improved filtering and data preparation.")


GME:
   Features correlated with target (>|0.1|): ['commonStockSharesOutstanding', 'Reported EPS', 'otherCurrentAssets', 'changeInOperatingLiabilities', 'changeInReceivables', 'profitLoss', 'netIncome', 'comprehensiveIncomeNetOfTax', 'totalShareholderEquity', 'interestExpense', 'netIncomeFromContinuingOperations', 'changeInOperatingAssets', 'sellingGeneralAndAdministrative', 'incomeBeforeTax', 'changeInInventory', 'ebit', 'totalLiabilities', 'operatingIncome', 'ebitda', 'operatingExpenses', 'otherNonCurrentAssets', 'costofGoodsAndServicesSold']
   Dropping 'ebit' with VIF = 9948.14
   Dropping 'incomeBeforeTax' with VIF = 924.77
   Dropping 'profitLoss' with VIF = 477.42
   Dropping 'ebitda' with VIF = 130.73
   Dropping 'netIncome' with VIF = 46.95
   Dropping 'operatingIncome' with VIF = 23.80
   Dropping 'changeInInventory' with VIF = 10.90
   Final independent features after VIF filtering: ['commonStockSharesOutstanding', 'otherCurrentAssets', 'changeInOperatingLiabilities', 'chan

/var/folders/cg/lqjpnn456tdcqmv7z04c7lch0000gn/T/ipykernel_79329/3238028082.py:110: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  reduced_data['Reported EPS'] = reduced_data['Reported EPS'].fillna(method='ffill').fillna(method='bfill')
/opt/anaconda3/lib/python3.12/site-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)
/opt/anaconda3/lib/python3.12/site-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)
/var/folders/cg/lqjpnn456tdcqmv7z04c7lch0000gn/T/ipykernel_79329/3238028082.py:110: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  reduced_data['Reported EPS'] = reduced_data['Reported EPS'].fillna(method='ffill').fillna(meth

   Dropping 'grossProfit' with VIF = inf
   Dropping 'depreciationAndAmortization' with VIF = inf
   Dropping 'netIncomeFromContinuingOperations' with VIF = inf
   Dropping 'incomeBeforeTax' with VIF = 713272034743.51
   Dropping 'totalRevenue' with VIF = 3204250439.96


/opt/anaconda3/lib/python3.12/site-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)
/opt/anaconda3/lib/python3.12/site-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


   Dropping 'intangibleAssets' with VIF = 243544944.56
   Dropping 'totalAssets' with VIF = 176234280.45
   Dropping 'netIncome' with VIF = 77533724.72
   Dropping 'operatingIncome' with VIF = 19339563.50
   Dropping 'comprehensiveIncomeNetOfTax' with VIF = 1359093.34
   Dropping 'totalCurrentAssets' with VIF = 620344.57
   Dropping 'ebitda' with VIF = 307514.00
   Dropping 'totalShareholderEquity' with VIF = 40590.83
   Dropping 'totalLiabilities' with VIF = 19226.71
   Dropping 'ebit' with VIF = 18711.93
   Dropping 'totalNonCurrentAssets' with VIF = 7757.70
   Dropping 'researchAndDevelopment' with VIF = 4541.52
   Dropping 'inventory' with VIF = 3032.95
   Dropping 'costOfRevenue' with VIF = 2498.05
   Dropping 'goodwill' with VIF = 1846.27
   Dropping 'shortLongTermDebtTotal' with VIF = 1125.63
   Dropping 'sellingGeneralAndAdministrative' with VIF = 1064.46
   Dropping 'propertyPlantEquipment' with VIF = 937.06
   Dropping 'otherCurrentLiabilities' with VIF = 615.83
   Dropping '

/var/folders/cg/lqjpnn456tdcqmv7z04c7lch0000gn/T/ipykernel_79329/3238028082.py:110: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  reduced_data['Reported EPS'] = reduced_data['Reported EPS'].fillna(method='ffill').fillna(method='bfill')
/opt/anaconda3/lib/python3.12/site-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)
/var/folders/cg/lqjpnn456tdcqmv7z04c7lch0000gn/T/ipykernel_79329/3238028082.py:110: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  reduced_data['Reported EPS'] = reduced_data['Reported EPS'].fillna(method='ffill').fillna(method='bfill')


In [36]:
reg_data = pd.read_csv("regression_data/GS_regression_data.csv")

reg_data.head()

,Surprise(%),netIncome,comprehensiveIncomeNetOfTax,incomeTaxExpense,EPS Estimate,longTermDebt,costOfRevenue,propertyPlantEquipment,paymentsForRepurchaseOfCommonStock,intangibleAssets,...,commonStock,totalNonCurrentAssets,netInterestIncome,grossProfit,totalCurrentAssets,intangibleAssetsExcludingGoodwill,paymentsForRepurchaseOfEquity,longTermDebtNoncurrent,depreciationAndAmortization,Shift_Reported_EPS
0,3.30,3043000000,3460000000,873000000,8.34,234632000000,8815000000,9729000000,3500000000,6885000000,...,9000000,158175000000,2242000000,8810000000,220383000000,992000000,4200000000,88360000000,646000000,8.40
1,35.28,4132000000,3733000000,1105000000,8.56,233919000000,8976000000,10514000000,1500000000,6918000000,...,9000000,159683000000,1608000000,9928000000,223068000000,1021000000,1500000000,87142000000,627000000,8.62
2,56.02,2008000000,1689000000,246000000,3.51,241877000000,9064000000,11244000000,1000000000,7093000000,...,9000000,155388000000,1339000000,6247000000,255324000000,1177000000,1000000000,86410000000,780000000,11.58
3,2.98,2058000000,2696000000,698000000,5.31,289341000000,9061000000,13138000000,1500000000,7254000000,...,9000000,157577000000,1547000000,7517000000,253621000000,1341000000,2500000000,73916000000,1512000000,5.48
4,-3.10,1216000000,580000000,520000000,3.18,295484000000,9159000000,-13650000000,750000000,7863000000,...,9000000,129655000000,1684000000,5444000000,284689000000,1921000000,750000000,74312000000,1594000000,5.47


## TASK 4: Conducting Multi-Variate Regression for all Stocks and making their own regression models 

In [37]:
import os
import pandas as pd
import numpy as np
import statsmodels.api as sm


stock_symbols = ["GME", "GS", "MSFT", "NVDA", "GOOGL"]

# Directories for regression data (from TASK 3) and final finance data (from TASK 2)
regression_dir = "regression_data"
final_dir = "final_finance_data"

predictions_list = []

for symbol in stock_symbols:
    print(f"\nProcessing prediction for {symbol}...")
    reg_file = os.path.join(regression_dir, f"{symbol}_regression_data.csv")
    
    # Load the regression dataset (created in TASK 3)
    try:
        reg_data = pd.read_csv(reg_file)
    except Exception as e:
        print(f"Error reading regression file for {symbol}: {e}")
        continue

    
    if "Shift_Reported_EPS" not in reg_data.columns:
        print(f"{symbol}: 'Shift_Reported_EPS' not found. Skipping.")
        continue

    
    Y = reg_data["Shift_Reported_EPS"]
    X = reg_data.drop(columns=["Shift_Reported_EPS"])
    
    if "Surprise(%)" in X.columns:
        X = X.drop(columns=["Surprise(%)"])
        print(f"   Removed 'Surprise(%)' from predictors for {symbol} for prediction consistency.")

    X_sm = sm.add_constant(X, has_constant='add')
    model = sm.OLS(Y, X_sm).fit()
    
    print(f"\nOLS Regression Summary for {symbol}:\n")
    print(model.summary())
    
    
    final_file = os.path.join(final_dir, f"{symbol}_final_finance_data.csv")
    try:
        final_df = pd.read_csv(final_file)
    except Exception as e:
        print(f"Error reading final finance file for {symbol}: {e}")
        continue
    
    
    if "date" in final_df.columns:
        final_df["date"] = pd.to_datetime(final_df["date"], errors='coerce')
        final_df.sort_values(by="date", ascending=False, inplace=True)
        earnings_date = final_df.iloc[0]["date"]
    else:
        print(f"Warning: 'date' column not found for {symbol} in final finance data.")
        earnings_date = pd.NaT

    # For predictor matching, drop the "date" column from final_df so it doesn't interfere.
    if "date" in final_df.columns:
        final_df = final_df.drop(columns=["date"])
    
    
    predictor_cols = X.columns.tolist()
    
    try:
        newest_row = final_df.iloc[[0]].copy()  
    except Exception as e:
        print(f"Error extracting newest row for {symbol}: {e}")
        continue

    
    if "Reported EPS" in newest_row.columns:
        actual_reported_eps = newest_row["Reported EPS"].values[0]
    else:
        actual_reported_eps = np.nan

    # Ensure newest_row contains all predictor columns.
    # For any predictor missing from newest_row, add it with a default value (mean from reg_data if available, else 0)
    for col in predictor_cols:
        if col not in newest_row.columns:
            default_val = reg_data[col].mean() if col in reg_data.columns else 0
            newest_row[col] = default_val
            print(f"   Added missing predictor '{col}' with default value {default_val:.2f} for {symbol}.")

    try:
        X_new = newest_row[predictor_cols]
    except Exception as e:
        print(f"Error selecting predictor columns for {symbol}: {e}")
        continue

    
    X_new_sm = sm.add_constant(X_new, has_constant='add')
    
    if X_new_sm.shape[1] != len(model.params):
        print(f"Error: For {symbol}, model expects {len(model.params)} columns but new data has {X_new_sm.shape[1]}. Skipping prediction.")
        continue

    
    predicted_eps = model.predict(X_new_sm).values[0]
    print(f"{symbol}: Predicted EPS = {predicted_eps:.2f}, Actual Reported EPS = {actual_reported_eps}")
    
    predictions_list.append({
        "Symbol": symbol,
        "Earnings_Date": earnings_date,
        "Predicted_EPS": predicted_eps,
        "Actual_Reported_EPS": actual_reported_eps
    })

# Create a DataFrame of all predictions and save it
predictions_df = pd.DataFrame(predictions_list)
output_file = "regression_predictions.csv"
predictions_df.to_csv(output_file, index=False)
print("\nSaved regression_predictions.csv:")
print(predictions_df)


Processing prediction for GME...
   Removed 'Surprise(%)' from predictors for GME for prediction consistency.

OLS Regression Summary for GME:

                            OLS Regression Results                            
Dep. Variable:     Shift_Reported_EPS   R-squared:                       0.676
Model:                            OLS   Adj. R-squared:                  0.576
Method:                 Least Squares   F-statistic:                     6.718
Date:                Fri, 04 Apr 2025   Prob (F-statistic):           4.35e-07
Time:                        12:26:29   Log-Likelihood:                 31.582
No. Observations:                  60   AIC:                            -33.16
Df Residuals:                      45   BIC:                            -1.748
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                                        coef    std err          

### The below code cell is used to make predicitons for all stocks for all the earnings dates as we need that to complete the backtest

In [38]:
import os
import pandas as pd
import numpy as np
import statsmodels.api as sm


stock_symbols = ["GME", "GS", "MSFT", "NVDA", "GOOGL"]

# Directories for regression data (from TASK 3) and final finance data (from TASK 2)
regression_dir = "regression_data"
final_dir = "final_finance_data"

predictions_list = []

for symbol in stock_symbols:
    print(f"\nProcessing predictions for {symbol}...")
    reg_file = os.path.join(regression_dir, f"{symbol}_regression_data.csv")
    
    # Load the regression dataset (created in TASK 3)
    try:
        reg_data = pd.read_csv(reg_file)
    except Exception as e:
        print(f"Error reading regression file for {symbol}: {e}")
        continue

    if "Shift_Reported_EPS" not in reg_data.columns:
        print(f"{symbol}: 'Shift_Reported_EPS' not found. Skipping.")
        continue

    Y = reg_data["Shift_Reported_EPS"]
    X = reg_data.drop(columns=["Shift_Reported_EPS"])
    
    if "Surprise(%)" in X.columns:
        X = X.drop(columns=["Surprise(%)"])
        print(f"   Removed 'Surprise(%)' from predictors for {symbol} for prediction consistency.")

    X_sm = sm.add_constant(X, has_constant='add')
    model = sm.OLS(Y, X_sm).fit()
    
    print(f"\nOLS Regression Summary for {symbol}:\n")
    print(model.summary())
    
    final_file = os.path.join(final_dir, f"{symbol}_final_finance_data.csv")
    try:
        final_df = pd.read_csv(final_file)
    except Exception as e:
        print(f"Error reading final finance file for {symbol}: {e}")
        continue
    
    # Use the "date" column exclusively as the earnings date
    if "date" in final_df.columns:
        final_df["date"] = pd.to_datetime(final_df["date"], errors='coerce')
        final_df.sort_values(by="date", ascending=True, inplace=True)
    else:
        print(f"Warning: 'date' column not found for {symbol} in final finance data.")
        continue  
    
    # We'll keep the "date" column in the final predictions output.
    # Do not drop it now.
    
   
    predictor_cols = X.columns.tolist()

    symbol_predictions = []
    
    for idx, row in final_df.iterrows():
        row_data = row.copy()
        for col in predictor_cols:
            if col not in row_data or pd.isnull(row_data[col]):
                default_val = reg_data[col].mean() if col in reg_data.columns else 0
                row_data[col] = default_val
                print(f"   [{symbol}] Added missing predictor '{col}' with default value {default_val:.2f} for row {idx}.")
        try:
            
            X_new = pd.DataFrame([row_data[predictor_cols]])
        except Exception as e:
            print(f"Error selecting predictor columns for {symbol} at row {idx}: {e}")
            continue
        
        # Add constant
        X_new_sm = sm.add_constant(X_new, has_constant='add')
        if X_new_sm.shape[1] != len(model.params):
            print(f"Skipping row {idx} for {symbol}: model expects {len(model.params)} columns but new data has {X_new_sm.shape[1]}.")
            continue
        
        
        pred_eps = model.predict(X_new_sm).values[0]
        
        actual_rep_eps = row_data.get("Reported EPS", np.nan)
        earnings_date = row_data.get("date", pd.NaT)
        symbol_predictions.append({
            "Symbol": symbol,
            "Earnings_Date": earnings_date,
            "Predicted_EPS": pred_eps,
            "Actual_Reported_EPS": actual_rep_eps
        })
    
    print(f"{symbol}: Processed {len(symbol_predictions)} predictions.")
    predictions_list.extend(symbol_predictions)

# Create a DataFrame of all predictions and save it
predictions_df = pd.DataFrame(predictions_list)
output_file = "regression_predictions_new.csv"
predictions_df.to_csv(output_file, index=False)
print("\nSaved regression_predictions_new.csv:")
print(predictions_df)


Processing predictions for GME...
   Removed 'Surprise(%)' from predictors for GME for prediction consistency.

OLS Regression Summary for GME:

                            OLS Regression Results                            
Dep. Variable:     Shift_Reported_EPS   R-squared:                       0.676
Model:                            OLS   Adj. R-squared:                  0.576
Method:                 Least Squares   F-statistic:                     6.718
Date:                Fri, 04 Apr 2025   Prob (F-statistic):           4.35e-07
Time:                        12:26:36   Log-Likelihood:                 31.582
No. Observations:                  60   AIC:                            -33.16
Df Residuals:                      45   BIC:                            -1.748
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                                        coef    std err         